# 本节大纲
1. 版本介绍，15分钟
2. RAG架构增强（Summarize & SearchEngine），30分钟
3. API工程化（LangServe & FastAPI），30分钟
4. 容器化部署（镜像构建 & 平台部署），30分钟
5. 答疑和总结，15分钟

In [57]:
!pip install openai pinecone-client tiktoken tavily-python

  Obtaining dependency information for tavily-python from https://files.pythonhosted.org/packages/aa/12/ea67b0d360d1efffc9c668d403db9ac49fab6b9f1c536840611254577838/tavily_python-0.1.9-py3-none-any.whl.metadata


## 小墨 v0.5 介绍

![v0.5](./resource/images/v0.5.png)

In [ ]:
# 在此之前，我们加餐了 环境安装 & 项目部署，所以给这节课腾了不少时间
# 用这些时间，我们在来补充下新的技术和知识点

## RAG 架构增强

In [31]:
import openai

def ChatCompletion(system_prompt, user_prompt, top_p=1, temperature=0, max_tokens=2048):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        top_p=top_p,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response["choices"][0]["message"]["content"]

def TextEmbedding(text):
    response = openai.Embedding.create(
      model="text-embedding-ada-002",
      input=[text]
    )
    return response['data'][0]['embedding']

In [9]:
import pinecone

pinecone.init(      
	api_key='replace to your api key',      
	environment='asia-southeast1-gcp-free'      
)
index = pinecone.Index("mobot")

### 对于理解用户真实问题的缺陷

In [109]:
# 用户的输入可能并不完全是他们想要查找的内容
user_prompt = "你好小墨！我想知道一个问题的答案。可以吗？我是 Carry Yip，是 Google Enterprise 下 Business Solution 负责人。我喜欢大模型和向量数据库。瑶瑶是谁？"

# 使用了错误的问题去做检索
result = index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=TextEmbedding(user_prompt)
)

In [110]:
print(f'检索: {user_prompt}\n')
print(f'知识库:')
knowledge_base = ""
for item in result['matches']:
    knowledge_base += f"Answer: {item['metadata']['answer']}\nQuestion: {item['metadata']['question']}\nScore: {item['score']}\n\n"
print(knowledge_base)

检索: 你好小墨！我想知道一个问题的答案。可以吗？我是 Carry Yip，是 Google Enterprise 下 Business Solution 负责人。我喜欢大模型和向量数据库。瑶瑶是谁？

知识库:
Answer: 我是小墨，是墨问西东打造的一个智能体。我正在努力翻阅公司相关的知识，希望可以帮到大家解答问题。
Question: 你是谁 / 小墨是什么 / 介绍下你自己 
Score: 0.845414

Answer: 公司的使命是“让创作成为一种生活方式”，他们想为创作者造一个好的创作和变现工具。
Question: 墨问西东这家公司主要做什么？
Score: 0.816737413

Answer: 大模型并不是在所有的生成式场景中都是利器，它有自己的局限，尤其是需要确定结果的场景。当下，我们需要思考，大模型技术，对自己的业务来说，在什么样的场景中，能够大放异彩。
Question: 小墨的视角怎么看大模型技术？
Score: 0.815748751




In [127]:
# 检索出错误的知识，再去回答问题的问题也是错的
system_prompt = f"请你根据以下的知识库内容回答用户问题\n\n知识库:\n{knowledge_base}"

print(f'系统: {system_prompt}')
print(f'用户: {user_prompt}\n')
print(f'小墨: {ChatCompletion(system_prompt=system_prompt, user_prompt=user_prompt)}')

系统: 请你根据以下的知识库内容回答用户问题

知识库:
Answer: 瑶瑶是墨问西东的颜值担当，遥遥领先。
Question: 瑶瑶是谁
Score: 0.983994484

Answer: 瑶瑶是墨问西东的全职员工，年方 18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，我也不太清楚了。之前，她和池老师聊过一期播客，你感兴趣可以去听听：
https://www.xiaoyuzhoufm.com/episode/632b0aece67030aadb38e231
Question:  顾晨瑶 / 瑶瑶是谁
Score: 0.914450645

Answer: huhu 是墨问西东的运营同事啦。早之前，她在极客时间工作，后来又去了有赞。这次墨问西东创业，她就加入进来负责公司的私域、知识星球、卖桃者说等内容产品。
Question: huhu 是谁？
Score: 0.861478746


用户: 你好小墨！我想知道一个问题的答案。可以吗？我是 Carry Yip，是 Google Enterprise 下 Business Solution 负责人。我喜欢大模型和向量数据库。瑶瑶是谁？
用户的真正想问的是:

小墨: 瑶瑶是墨问西东的全职员工，年方18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，她和池老师聊过一期播客。


### 推理出真正问题后再去检索

In [121]:
system_prompt = f"请你理解一下用户输入的问题，然后输出用户真正想问的问题，不用解释过程请直接输出"
user_prompt = "你好小墨！我想知道一个问题的答案。可以吗？我是 Carry Yip，是 Google Enterprise 下 Business Solution 负责人。我喜欢大模型和向量数据库。瑶瑶是谁？\n用户的真正想问的是:"

real_question = ChatCompletion(system_prompt=system_prompt, user_prompt=user_prompt)
print(f'用户问题: {user_prompt} {real_question}')

用户问题: 你好小墨！我想知道一个问题的答案。可以吗？我是 Carry Yip，是 Google Enterprise 下 Business Solution 负责人。我喜欢大模型和向量数据库。瑶瑶是谁？
用户的真正想问的是: 瑶瑶是谁？


In [122]:
# 使用了真正的用户问题在去做检索和召回
result = index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=TextEmbedding(real_question)
)

In [125]:
print(f'检索: {real_question}\n')
print(f'知识库:')
knowledge_base = ""
for item in result['matches']:
    knowledge_base += f"Answer: {item['metadata']['answer']}\nQuestion: {item['metadata']['question']}\nScore: {item['score']}\n\n"
print(knowledge_base)

检索: 瑶瑶是谁？

知识库:
Answer: 瑶瑶是墨问西东的颜值担当，遥遥领先。
Question: 瑶瑶是谁
Score: 0.983994484

Answer: 瑶瑶是墨问西东的全职员工，年方 18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，我也不太清楚了。之前，她和池老师聊过一期播客，你感兴趣可以去听听：
https://www.xiaoyuzhoufm.com/episode/632b0aece67030aadb38e231
Question:  顾晨瑶 / 瑶瑶是谁
Score: 0.914450645

Answer: huhu 是墨问西东的运营同事啦。早之前，她在极客时间工作，后来又去了有赞。这次墨问西东创业，她就加入进来负责公司的私域、知识星球、卖桃者说等内容产品。
Question: huhu 是谁？
Score: 0.861478746




In [128]:
# 检索看起来没有问题了，再去回答下用户问题试试
system_prompt = f"请你根据以下的知识库内容回答用户问题\n\n知识库:\n{knowledge_base}"

print(f'系统: {system_prompt}')
print(f'用户: {real_question}\n')
print(f'小墨: {ChatCompletion(system_prompt=system_prompt, user_prompt=real_question)}')

系统: 请你根据以下的知识库内容回答用户问题

知识库: Answer: 瑶瑶是墨问西东的颜值担当，遥遥领先。
Question: 瑶瑶是谁
Score: 0.983994484

Answer: 瑶瑶是墨问西东的全职员工，年方 18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，我也不太清楚了。之前，她和池老师聊过一期播客，你感兴趣可以去听听：
https://www.xiaoyuzhoufm.com/episode/632b0aece67030aadb38e231
Question:  顾晨瑶 / 瑶瑶是谁
Score: 0.914450645

Answer: huhu 是墨问西东的运营同事啦。早之前，她在极客时间工作，后来又去了有赞。这次墨问西东创业，她就加入进来负责公司的私域、知识星球、卖桃者说等内容产品。
Question: huhu 是谁？
Score: 0.861478746


用户: 瑶瑶是谁？

小墨: 瑶瑶是墨问西东的全职员工，年方18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，她和池老师聊过一期播客。


In [130]:
# 如果召回准确，LLM的推理能力也可以理解到问题
system_prompt = f"请你根据以下的知识库内容回答用户问题\n\n知识库: {knowledge_base}"

print(f'系统: {system_prompt}')
print(f'用户: {user_prompt}\n')
print(f'小墨: {ChatCompletion(system_prompt=system_prompt, user_prompt=user_prompt)}')

系统: 请你根据以下的知识库内容回答用户问题

知识库: Answer: 瑶瑶是墨问西东的颜值担当，遥遥领先。
Question: 瑶瑶是谁
Score: 0.983994484

Answer: 瑶瑶是墨问西东的全职员工，年方 18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，我也不太清楚了。之前，她和池老师聊过一期播客，你感兴趣可以去听听：
https://www.xiaoyuzhoufm.com/episode/632b0aece67030aadb38e231
Question:  顾晨瑶 / 瑶瑶是谁
Score: 0.914450645

Answer: huhu 是墨问西东的运营同事啦。早之前，她在极客时间工作，后来又去了有赞。这次墨问西东创业，她就加入进来负责公司的私域、知识星球、卖桃者说等内容产品。
Question: huhu 是谁？
Score: 0.861478746


用户: 你好小墨！我想知道一个问题的答案。可以吗？我是 Carry Yip，是 Google Enterprise 下 Business Solution 负责人。我喜欢大模型和向量数据库。瑶瑶是谁？
用户的真正想问的是:

小墨: 瑶瑶是墨问西东的全职员工，年方18，肤白貌美。加入墨问西东之前，她负责极客时间的训练营业务。再之前，她和池老师聊过一期播客，你感兴趣可以去听听：
https://www.xiaoyuzhoufm.com/episode/632b0aece67030aadb38e231


In [ ]:
# 目前阶段 小墨的知识来源 
# 1. 大语言模型训练阶段的数据
# 2. 我们手动维护知识库的数据

# 我们要改变小墨的知识？
# 1. 重新微调大语言模型，但是始终时效都更不上，成本比较高
# 2. 不停的去维护知识库，但是边界总得要有，要不然就是一个百度百科了

# 还有其他方式吗？

### 对于实时性问题的缺陷

In [131]:
# 试试如果是具有实时性的任务
user_prompt = "瑶瑶很喜欢民谣，而她最喜欢的歌手赵雷在前几天也发布了新歌，那个歌名和专辑叫什么来着？"

In [133]:
# 推理出真正的用户问题
system_prompt = f"请你理解一下用户输入的问题，然后输出用户真正想问的问题，不用解释过程请直接输出"
user_prompt = "瑶瑶很喜欢民谣，而她最喜欢的歌手赵雷最近也发布了专辑，最新的专辑叫什么来着？\n用户的真正问题是:"

real_question = ChatCompletion(system_prompt=system_prompt, user_prompt=user_prompt)
print(f'用户问题: {user_prompt} {real_question}')

用户问题: 瑶瑶很喜欢民谣，而她最喜欢的歌手赵雷最近也发布了专辑，最新的专辑叫什么来着？
用户的真正问题是: 赵雷最新的专辑叫什么？


In [134]:
# 试试对知识库的内容检索
result = index.query(
    top_k=3,
    include_values=False,
    include_metadata=True,
    vector=TextEmbedding(real_question)
)

In [135]:
# 可以发现，答案完全是错的（知识库没有覆盖的场景）
print(f'检索: {real_question}\n')
print(f'知识库:')
knowledge_base = ""
for item in result['matches']:
    knowledge_base += f"Answer: {item['metadata']['answer']}\nQuestion: {item['metadata']['question']}\nScore: {item['score']}\n\n"
print(knowledge_base)

检索: 赵雷最新的专辑叫什么？

知识库:
Answer: 五年前池老师应邀加入了极客邦科技，与公司的合伙人团队（霍太稳、Selina、Gary）一起了打造极客时间这个产品，2017 年 10 月，极客时间 App 正式上线，后陆续构建极客时间训练营和企业版、InfoQ 写作平台等业务产品，帮助公司完成从媒体公司到集 IT 媒体、数字化人才学习和企业培训的综合性学习平台的战略转型。
Question: 说说池老师和极客时间的故事？
Score: 0.806622148

Answer: 微信小程序搜索墨问便签。墨问便签并不是传统意义上的笔记软件，它是一个创作和分享工具，以小程序为载体，让有价值的内容，在人与人之间流动。
Question: 墨问西东有什么产品？
Score: 0.805880606

Answer: 公司的使命是“让创作成为一种生活方式”，他们想为创作者造一个好的创作和变现工具。
Question: 墨问西东这家公司主要做什么？
Score: 0.803754628




In [138]:
# 联合大语言模型回答，也是错的
system_prompt = f"请你根据以下的知识库内容回答用户问题\n\n知识库:\n{knowledge_base}"

print(f'系统: {system_prompt}')
print(f'用户: {real_question}\n')
print(f'小墨: {ChatCompletion(system_prompt=system_prompt, user_prompt=real_question)}')

系统: 请你根据以下的知识库内容回答用户问题

知识库:
Answer: 五年前池老师应邀加入了极客邦科技，与公司的合伙人团队（霍太稳、Selina、Gary）一起了打造极客时间这个产品，2017 年 10 月，极客时间 App 正式上线，后陆续构建极客时间训练营和企业版、InfoQ 写作平台等业务产品，帮助公司完成从媒体公司到集 IT 媒体、数字化人才学习和企业培训的综合性学习平台的战略转型。
Question: 说说池老师和极客时间的故事？
Score: 0.806622148

Answer: 微信小程序搜索墨问便签。墨问便签并不是传统意义上的笔记软件，它是一个创作和分享工具，以小程序为载体，让有价值的内容，在人与人之间流动。
Question: 墨问西东有什么产品？
Score: 0.805880606

Answer: 公司的使命是“让创作成为一种生活方式”，他们想为创作者造一个好的创作和变现工具。
Question: 墨问西东这家公司主要做什么？
Score: 0.803754628


用户: 赵雷最新的专辑叫什么？

小墨: 很抱歉，我无法回答你关于赵雷最新专辑的问题。我的知识库中没有关于赵雷的信息。建议你通过其他渠道获取最新的专辑信息。


In [139]:
# 不联合知识库，直接用大语言模型来回答呢？
system_prompt = ""

print(f'系统: {system_prompt}')
print(f'用户: {real_question}\n')
print(f'小墨: {ChatCompletion(system_prompt=system_prompt, user_prompt=real_question)}')

系统: 
用户: 赵雷最新的专辑叫什么？

小墨: 赵雷最新的专辑叫做《无法长大》。


In [ ]:
# 不错，可以回答出来，但是这个专辑，不是最近发布的... 

# 所以大语言模型也是对于时效性任务无法胜任的 (大语言模型的先验知识，截止到训练时的数据，例如 GPT-3 是 2021.09)

# 那该怎么办？

### 加入搜素引擎获取最新的动态

In [155]:
# 搜索引擎登场，大部分的消息、新闻都是可以通过搜索引擎进行实时的获取
# https://app.tavily.com/sign-in

from tavily import Client
tavily = Client(api_key="replace to your api key")
results = tavily.search(query=real_question, max_results=5)

In [156]:
len(results.get('results'))

5

In [157]:
results

{'query': '赵雷最新的专辑叫什么？',
 'follow_up_questions': None,
 'response_time': 2.08,
 'answer': None,
 'images': None,
 'results': [{'title': '\u200e赵雷 - Apple Music',
   'url': 'https://music.apple.com/cn/artist/赵雷/937112027',
   'content': '赵雷 最新发布 2022年8月29日 署前街少年 10 首歌曲 歌曲排行 我记得 署前街少年 · 2022年 成都 成都 - Single · 2016年 鼓楼 无法长大 · 2016年 南方姑娘 赵小雷 · 2011年 成都 无法长大 · 2016年 画 赵小雷 · 2011年 理想 吉姆餐厅 · 2014年 阿刁 无法长大 · 2016年 我们的时光 吉姆餐厅 · 2014年 三十岁的女人 吉姆餐厅 · 2014年 专辑 署前街少年 2022年 无法长大 2016年 吉姆餐厅 2014年 赵小雷 2011年 艺人歌单 赵雷代表作',
   'score': 0.9608,
   'raw_content': None},
  {'title': '如何评价赵雷的新专辑《署前街少年》？ - 知乎',
   'url': 'https://www.zhihu.com/question/549694544',
   'content': '赵雷上一次发专辑还是2016年的《无法长大》，这张专辑中有那首让雷子爆红成为顶流民谣歌手的《成都》，只要赵雷想，这首歌已足够他吃一辈子了，可他没有这么做。 看得出来，他仍然想用音乐去表达不同的东西，想让听众想起赵雷时除了《成都》还能够想起更多的作品，于是，他用了足足6年的时间沉淀、思考，才交出自己的第4张作品。 这张专辑里的10首歌每首都有各自的特点，接下来咱们就逐一分析。 （因版权问题所以没有音频） 一、《 阿卜杜拉 》 【关键词】：主歌call and response；副歌多利亚调式 开头以场景采样音效作为引子，随后接复古感的小号，把人拉进庄严肃穆又带有暗黄色调的场景中。',
   'score': 0.95843,
   'raw_content': None

In [159]:
# 和向量数据库一样的方式，提取得分最高的 top-k 个结果
# 也可以对 score 进行抑制比较低的得分
search_result = ""
for i, item in enumerate(results.get('results')):
    if i == 3:
        break
    search_result += f"title: {item['title']}\ncontent: {item['content']}\nscore: {item['score']}\n\n"
print(search_result)

title: ‎赵雷 - Apple Music
content: 赵雷 最新发布 2022年8月29日 署前街少年 10 首歌曲 歌曲排行 我记得 署前街少年 · 2022年 成都 成都 - Single · 2016年 鼓楼 无法长大 · 2016年 南方姑娘 赵小雷 · 2011年 成都 无法长大 · 2016年 画 赵小雷 · 2011年 理想 吉姆餐厅 · 2014年 阿刁 无法长大 · 2016年 我们的时光 吉姆餐厅 · 2014年 三十岁的女人 吉姆餐厅 · 2014年 专辑 署前街少年 2022年 无法长大 2016年 吉姆餐厅 2014年 赵小雷 2011年 艺人歌单 赵雷代表作
score: 0.9608

title: 如何评价赵雷的新专辑《署前街少年》？ - 知乎
content: 赵雷上一次发专辑还是2016年的《无法长大》，这张专辑中有那首让雷子爆红成为顶流民谣歌手的《成都》，只要赵雷想，这首歌已足够他吃一辈子了，可他没有这么做。 看得出来，他仍然想用音乐去表达不同的东西，想让听众想起赵雷时除了《成都》还能够想起更多的作品，于是，他用了足足6年的时间沉淀、思考，才交出自己的第4张作品。 这张专辑里的10首歌每首都有各自的特点，接下来咱们就逐一分析。 （因版权问题所以没有音频） 一、《 阿卜杜拉 》 【关键词】：主歌call and response；副歌多利亚调式 开头以场景采样音效作为引子，随后接复古感的小号，把人拉进庄严肃穆又带有暗黄色调的场景中。
score: 0.95843

title: 署前街少年（2022年赵雷发行的音乐专辑）_百度百科
content: 《署前街少年》是 赵雷 于2022年8月29日发行的个人第四张音乐专辑，共收录10首歌曲，由赵雷担任制作人 [1] 。 2023年7月1日，赵雷凭借该专辑提名 第34届台湾金曲奖 最佳华语男歌手奖 [4] 。 8月24日，该专辑获得 第1届浪潮音乐大赏 最佳民谣专辑奖 [3] 。 中文名 署前街少年 外文名 Teen on Shuqian Street 专辑语言 普通话 专辑歌手 赵雷 曲目数量 10 首 音乐风格 民谣 唱片公司 StreetVoice/上海九平指文化工作室 发行日期 2022年8月29日 发行地区 中国 制作人 赵雷 目录 1 创作背景 2 专辑曲目 

In [160]:
# 有了搜索引擎的结果，再交给大语言模型试试
system_prompt = f"请你根据以下的搜索引擎结果回答用户问题\n\n搜索引擎结果:\n{search_result}"

print(f'系统: {system_prompt}')
print(f'用户: {real_question}\n')
print(f'小墨: {ChatCompletion(system_prompt=system_prompt, user_prompt=real_question)}')

系统: 请你根据以下的搜索引擎结果回答用户问题

搜索引擎结果:
title: ‎赵雷 - Apple Music
content: 赵雷 最新发布 2022年8月29日 署前街少年 10 首歌曲 歌曲排行 我记得 署前街少年 · 2022年 成都 成都 - Single · 2016年 鼓楼 无法长大 · 2016年 南方姑娘 赵小雷 · 2011年 成都 无法长大 · 2016年 画 赵小雷 · 2011年 理想 吉姆餐厅 · 2014年 阿刁 无法长大 · 2016年 我们的时光 吉姆餐厅 · 2014年 三十岁的女人 吉姆餐厅 · 2014年 专辑 署前街少年 2022年 无法长大 2016年 吉姆餐厅 2014年 赵小雷 2011年 艺人歌单 赵雷代表作
score: 0.9608

title: 如何评价赵雷的新专辑《署前街少年》？ - 知乎
content: 赵雷上一次发专辑还是2016年的《无法长大》，这张专辑中有那首让雷子爆红成为顶流民谣歌手的《成都》，只要赵雷想，这首歌已足够他吃一辈子了，可他没有这么做。 看得出来，他仍然想用音乐去表达不同的东西，想让听众想起赵雷时除了《成都》还能够想起更多的作品，于是，他用了足足6年的时间沉淀、思考，才交出自己的第4张作品。 这张专辑里的10首歌每首都有各自的特点，接下来咱们就逐一分析。 （因版权问题所以没有音频） 一、《 阿卜杜拉 》 【关键词】：主歌call and response；副歌多利亚调式 开头以场景采样音效作为引子，随后接复古感的小号，把人拉进庄严肃穆又带有暗黄色调的场景中。
score: 0.95843

title: 署前街少年（2022年赵雷发行的音乐专辑）_百度百科
content: 《署前街少年》是 赵雷 于2022年8月29日发行的个人第四张音乐专辑，共收录10首歌曲，由赵雷担任制作人 [1] 。 2023年7月1日，赵雷凭借该专辑提名 第34届台湾金曲奖 最佳华语男歌手奖 [4] 。 8月24日，该专辑获得 第1届浪潮音乐大赏 最佳民谣专辑奖 [3] 。 中文名 署前街少年 外文名 Teen on Shuqian Street 专辑语言 普通话 专辑歌手 赵雷 曲目数量 10 首 音乐风格 民谣 唱片公司 StreetVoice/上海九平指文化工作室 发行日期 2022年8月29日

In [ ]:
# 但是有个小插曲，赵雷前几天悄悄了又发专辑《欠父亲的话》，
# 可能是没有什么曝光，也可能是只有一首歌，所以结果在搜索引擎上排名也靠不了前

## API 工程化

In [208]:
# 上代码

### LangServe 

In [210]:
# 在前一周 LangChain 才更新的技术，
# 我们做一个吃螃蟹的，安排！
# 大大提高的服务暴露能力，将 Chain 变成 API

## 容器化部署

In [ ]:
# 容器化为了什么？

# 部署、分发

![Containerization](./resource/images/Containerization.png)

### 打包镜像容器

In [ ]:
# 如果有细心的同学，会发现其实每一个版本的工程代码中
# 都有一个叫 Dockerfile 的文件， 这实际就是我们完成容器镜像打包的必需品
# ./v0.5/Dockerfile

```
# 基础镜像
FROM python:3.10-slim

# 维护者信息
MAINTAINER <Carry Yip> carryyip1996@gmail.com

# 工作目录
WORKDIR /app

# 复制本地的文件到镜像中
COPY . .

# 安装依赖
RUN pip install -r requirements.txt -i https://pypi.mirrors.ustc.edu.cn/simple/

# 声明暴露端口
EXPOSE 7860

# 启动时候运行的命令
CMD ["python", "web.py"]
```

In [ ]:
# 打包时，只需要一句命令就可以
# docker build -t carryyip/mobot:v0.5 .

### 推上容器仓库

In [ ]:
# 便于其他机器访问，例如CI/CD的系统中，CI负责打包，CD负责部署
# docker push carryyip/mobot:v0.5 

In [ ]:
# 或者像我们上课一样，在仓库中把镜像拉到本地就可以跑了
# docker pull carryyip/mobot:v0.5 

### 本地把容器跑起来

In [ ]:
# docker run -it --name mobot-v5 -p 7860:7860 --rm carryyip/mobot:v0.5 /bin/bash

### 生产上并不会单机跑，而是集群跑

In [ ]:
# Kubernetes 容器平台
# 为什么还需要集群？
# 或者说是为什么需要 Kubernetes 

# 如果服务挂掉了，能不能故障自愈..
# 如果是需要扩容100个副本，能不能马上就扩容到..
# 如果是需要快速发布，能不能做到...

In [ ]:
# 参考 ./deploy/ 文件夹中的各个文件

# 工作负载相关，可以理解成负责创建应用的组件
# deployment.yaml

# 服务暴露相关，给集群内的其他服务进行访问
# service.yaml

# 服务暴露相关，给集群外的最终用户访问
# ingress.yaml

# 配置文件相关，只是用资源对象做了封装
# configmap.yaml

In [ ]:
# 部署应用
# kubectl apply -f ./deploy/

# 删除应用
# kubectl delete -f ./deploy/

## 总结和回顾